# Explore whether high dimensional lab values/ICD9 contribute to prediction models

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cmap
import scipy as sp
import pymc3 as pm
import pandas as pd
from sklearn import preprocessing
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score
import seaborn as sns
import theano
import theano.tensor as tt
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_auc_score

import itertools
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

We know large volumes of data are generated in hospital settings, including clinical and physiological data generated during the course of patient care. Since our goal was to identify early clinical factors or traits useful for predicting the readmission outcome, I want to explore whether including the high dimension lab values and ICD9 values will contribute to better prediction performance.
I start with some basic summarizing of lab values/ICD9 data while preserve the high dimension nature of these data. We know high dimension of variables can include lots of noise and be very slow to deal with. Then I try some variable selection to reduce the dimension of data and preserve some useful information within the data.

## get the patient ID and time window needed 
We need to predict the readmission event of certain patient, during certain time windows. So we get that informationusing ADT dataset.

In [ ]:
#import adt file, get time window 

infile9 = "/Users/ying/Dropbox/Data/FONNESBECK_ADT_20151202.csv"

df_adt = pd.read_csv(infile9, encoding = "ISO-8859-1")
df_adt = df_adt.loc[df_adt['Event'].isin(['Admit','Discharge'])]
df_adt.drop(['SRV_CODE', 'CHIEF_COMPLAINT', 'Event_Date'], axis = 1, inplace = True)
df_adt['Admission_date'] = pd.to_datetime(df_adt['Admission_date'])
df_adt['DISCHARGE_DATE'] = pd.to_datetime(df_adt['DISCHARGE_DATE'])
df_adt.sort_values(['RUID', 'Admission_date', 'DISCHARGE_DATE'], inplace=True)
df_adt.head()

def clean_table(df):
    res = []
    for id in df.RUID.unique():
        # get the temp dataset for this id
        temp = df[df.RUID == id]
        # more than 2 like admission discharge admission
        if len(temp) > 2:
            temp = temp.assign(start_date = None)
            temp = temp.assign(end_date = None)
            temp = temp.assign(readmission = None)
            temp['delta_days'] = (temp.Admission_date.shift(-1) - temp.DISCHARGE_DATE).dt.days
            temp['start_date'] = temp.Admission_date.shift(1)
            temp['end_date'] = temp.Admission_date.shift(-1)


            # only use ADT DISCHARGE row
            temp = temp[(temp['Event'] == 'Discharge') & temp['delta_days'] > 0]
            temp['readmission'] = (temp['delta_days'] <= 30).astype(int)
            temp.drop(['Event', 'Admission_date', 'DISCHARGE_DATE', 'delta_days'], axis = 1, inplace=True)

            # now we have temp table have columns: RUID, start_date, end_date, readmission
            # append it to res
            res.append(temp)
    return pd.concat(res)  

df = clean_table(df_adt)
df.head()

## Import lab values

The way to deal with lab values: using the mean "lab value" of each individual "lab name"
0) build a empty list to store results
1) use dftime to get ruid, start time and end time (time window of that patient)
2) use these information to subset the related lab events for this patient and during this time period, group by "lab name" and get mean of each group, store this in a list
3) concatenate all dataframes in the list


In [ ]:
#import lab file


infile5 = "/Users/ying/Dropbox/Data/FONNESBECK_LAB_20151202.csv"

df_lab = pd.read_csv(infile5, quoting=csv.QUOTE_NONE, encoding="ISO-8859-1")
df_lab['Lab_date'] = pd.to_datetime(df_lab['Lab_date'], errors='coerce')
df_lab = df_lab.sort_values(['RUID', 'Lab_date'])
df_lab['Lab_value'] = df_lab['Lab_value'].str.extract('(\d+)', expand=False).astype('float')

In [ ]:
#start a list to store the part of df_lab selected for each row in df

biglist=[]

for index, row in df.iterrows():    
    mask = (df_lab['RUID'] == row['RUID']) \
    & (df_lab['Lab_date'] >= row['start_date']) \
    & (df_lab['Lab_date'] <= row['end_date'])
    new_row=df_lab.loc[mask,]
    
    new1=new_row.groupby('Lab_name').mean()
    record=np.empty((1,new1.shape[0]))
    record[0,]=new1['Lab_value'].values
    new12=pd.DataFrame(record, columns=new1.index.tolist())
    
    biglist.append(new12)

In [ ]:
#summarize lab values
lablist=pd.concat(biglist,axis=0,join="outer")
lab=lablist.fillna(0) #25152 rows

## combine lab values data with all the other data 

In [ ]:
#combine with the clean dataset without individual lab values
data = pd.read_csv("/Users/ying/Bios8366/BIOS_8366_Proj_Rui_Gillian_Ying/Data/cleandata.csv")
data = data[['readmission', 'Sex', 'Race', 'age', 'Pregnancy_Indicator', 
             'BMI', 'SYSTOLIC', 'DIASTOLIC', 'EGFR', 'cpt_count', 'cpt_nunique', 
             'icd_count', 'icd_nunique', 'Lab_value', 'Lab_count', 'drug_count', 'drug_nunique']]

## setup predictors and outcome for modeling

In [ ]:
df=pd.concat([data,lab],axis=1) #data and lab have same row number, concatenate
#df shape: (21252, 2756)

In [ ]:
#follow Rui's work in step 3 for modeling



X = df.drop(['readmission'], axis = 1).values
y = df['readmission'].values

# replace the missing with median
X = Imputer(strategy = 'median').fit_transform(X)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

## basic modeling performance

In [ ]:
clfs = {'LogisticRegression':LogisticRegression(),'SVC': SVC(), 
        'KNeighborsClassifier': KNeighborsClassifier(n_neighbors = 3),
        'GaussianNB': GaussianNB(),
        'Perceptron': Perceptron(), 'LinearSVC': LinearSVC(), 
        'SGDClassifier': SGDClassifier(), 
        'DecisionTreeClassifier': DecisionTreeClassifier(),
        'RandomForestClassifier': RandomForestClassifier(n_estimators=100)}

for name, clf in clfs.items():
    clf.fit(X_train,y_train)
    ypred = clf.predict(X_test)
    precision_ = precision_score(ypred, y_test)
    recall_ = recall_score(ypred, y_test)
    accuracy_ = accuracy_score(ypred,y_test)
    f1_ = f1_score(ypred,y_test)
    print('%s classifier: precision = %.4f, recall = %.4f, accuracy = %.4f, f1 score = %.4f' 
          %(name, precision_, recall_, accuracy_, f1_))

#performance evaluation (output from above)

LogisticRegression classifier: precision = 0.6049, recall = 0.6364, accuracy = 0.7177, f1 score = 0.6203

SVC classifier: precision = 0.4626, recall = 0.6816, accuracy = 0.7128, f1 score = 0.5511

KNeighborsClassifier classifier: precision = 0.5226, recall = 0.5467, accuracy = 0.6529, f1 score = 0.5344

GaussianNB classifier: precision = 0.9617, recall = 0.4002, accuracy = 0.4362, f1 score = 0.5652

Perceptron classifier: precision = 0.5144, recall = 0.5677, accuracy = 0.6656, f1 score = 0.5397

LinearSVC classifier: precision = 0.6070, recall = 0.6208, accuracy = 0.7089, f1 score = 0.6138

SGDClassifier classifier: precision = 0.5486, recall = 0.5543, accuracy = 0.6598, f1 score = 0.5514

DecisionTreeClassifier classifier: precision = 0.5626, recall = 0.5508, accuracy = 0.6584, f1 score = 0.5566

RandomForestClassifier classifier: precision = 0.4819, recall = 0.7397, accuracy = 0.7379, f1 score = 0.5836

## variable selection of lab
Although this looks better than without lab results, we are concerned about using high dimensions of data.
First I wonder what features contribute more. To do it quick and simple, I use the "feature importance"in random forest.

### feature importance using random forest

In [ ]:
#what features are important?

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=4)
rf.fit(X_train, y_train)

preds = rf.predict(X_test)
pd.crosstab(y_test, preds, rownames=['actual'], 
            colnames=['prediction'])


prediction	0	1
actual		
0	3420	526
1	1372	1058

In [ ]:

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")

for f in range(df.shape[1]):
    print("%d. %s (%f)" % (f + 1, df.columns[indices[f]], importances[indices[f]]))

Feature ranking (selected top 20):
1. cpt_nunique (0.028876)
2. Lab_count (0.028844)
3. cpt_count (0.024849)
4. EGFR (0.023369)
5. HexacB (0.019920)
6. drug_count (0.019067)
7. PCP (0.018809)
Phencyclidine Screen, Urine Test (3532U)
8. RB1 (0.017485)
Retinoblastoma (RB1) Testing
9. icd_count (0.016888)
10. Pregnancy_Indicator (0.016818)
11. Lab_value (0.016127)
12. SYSTOLIC (0.014930)
13. BMI (0.014630)
14. Race (0.013513)
15. PlsHem (0.013206)
HEMOGLOBIN PLASMA
16. VtAInt (0.013064)
VITAMIN A INTERPRETATION
17. BPMPT (0.012998)
18. CYT-ST (0.011991)
Cytogenetics Studies
19. AMTSBH (0.011770)
AMOUNT SAMPLE  HELD
20. Alanin (0.011674)
ALANINE

### variable reduction using PCA

However, this was using all lab data. So our dataset was 21252 rows and 2756 columns. 
We want to get some dimension reduction for the following reasons:

1) It helps in data compressing and reducing the storage space required, now the data is >200 MB.

2) It fastens the time required for performing same computations. Less dimensions leads to less computing, also less dimensions can allow usage of algorithms unfit for a large number of dimensions

3) It removes redundant features

To start with, I tried a simple PCA

### use 100 PC from lab + clean data

In [ ]:


from sklearn.decomposition import PCA

pca = PCA(n_components=100, whiten=True).fit(lab)
lab_pca = pca.transform(lab)

lab3=pd.DataFrame(lab_pca)

df100=pd.concat([data,lab3],axis=1) #df100 shape: (21252, 117)

X = df100.drop(['readmission'], axis = 1).values
y = df100['readmission'].values

# replace the missing with median
X = Imputer(strategy = 'median').fit_transform(X)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
#modeling and evaluation

clfs = {'LogisticRegression':LogisticRegression(),'SVC': SVC(), 
        'KNeighborsClassifier': KNeighborsClassifier(n_neighbors = 3),
        'GaussianNB': GaussianNB(),
        'Perceptron': Perceptron(), 'LinearSVC': LinearSVC(), 
        'SGDClassifier': SGDClassifier(), 
        'DecisionTreeClassifier': DecisionTreeClassifier(),
        'RandomForestClassifier': RandomForestClassifier(n_estimators=100)}

for name, clf in clfs.items():
    clf.fit(X_train,y_train)
    ypred = clf.predict(X_test)
    precision_ = precision_score(ypred, y_test)
    recall_ = recall_score(ypred, y_test)
    accuracy_ = accuracy_score(ypred,y_test)
    f1_ = f1_score(ypred,y_test)
    print('%s classifier: precision = %.4f, recall = %.4f, accuracy = %.4f, f1 score = %.4f' 
          %(name, precision_, recall_, accuracy_, f1_))

#performance evaluated (output from above)

LogisticRegression classifier: precision = 0.3498, recall = 0.5886, accuracy = 0.6590, f1 score = 0.4388

SVC classifier: precision = 0.3123, recall = 0.6161, accuracy = 0.6637, f1 score = 0.4145

KNeighborsClassifier classifier: precision = 0.4835, recall = 0.5154, accuracy = 0.6299, f1 score = 0.4989

GaussianNB classifier: precision = 0.9778, recall = 0.3843, accuracy = 0.3944, f1 score = 0.5517

Perceptron classifier: precision = 0.6440, recall = 0.5128, accuracy = 0.6311, f1 score = 0.5710

LinearSVC classifier: precision = 0.2988, recall = 0.5893, accuracy = 0.6534, f1 score = 0.3965

SGDClassifier classifier: precision = 0.3222, recall = 0.4845, accuracy = 0.6110, f1 score = 0.3870

DecisionTreeClassifier classifier: precision = 0.5140, recall = 0.5081, accuracy = 0.6252, f1 score = 0.5110

RandomForestClassifier classifier: precision = 0.4379, recall = 0.6642, accuracy = 0.7014, f1 score = 0.5278

### use 20 PC from lab + clean data

In [ ]:


from sklearn.decomposition import PCA

pca = PCA(n_components=20, whiten=True).fit(lab)
lab_pca = pca.transform(lab)

lab2=pd.DataFrame(lab_pca)

df20=pd.concat([data,lab3],axis=1) #df20 shape: (21252, 36)

X = df20.drop(['readmission'], axis = 1).values
y = df20['readmission'].values

# replace the missing with median
X = Imputer(strategy = 'median').fit_transform(X)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
#modeling and evaluation

clfs = {'LogisticRegression':LogisticRegression(),'SVC': SVC(), 
        'KNeighborsClassifier': KNeighborsClassifier(n_neighbors = 3),
        'GaussianNB': GaussianNB(),
        'Perceptron': Perceptron(), 'LinearSVC': LinearSVC(), 
        'SGDClassifier': SGDClassifier(), 
        'DecisionTreeClassifier': DecisionTreeClassifier(),
        'RandomForestClassifier': RandomForestClassifier(n_estimators=100)}

for name, clf in clfs.items():
    clf.fit(X_train,y_train)
    ypred = clf.predict(X_test)
    precision_ = precision_score(ypred, y_test)
    recall_ = recall_score(ypred, y_test)
    accuracy_ = accuracy_score(ypred,y_test)
    f1_ = f1_score(ypred,y_test)
    print('%s classifier: precision = %.4f, recall = %.4f, accuracy = %.4f, f1 score = %.4f' 
          %(name, precision_, recall_, accuracy_, f1_))

LogisticRegression classifier: precision = 0.3498, recall = 0.5886, accuracy = 0.6590, f1 score = 0.4388

SVC classifier: precision = 0.3123, recall = 0.6161, accuracy = 0.6637, f1 score = 0.4145

KNeighborsClassifier classifier: precision = 0.4835, recall = 0.5154, accuracy = 0.6299, f1 score = 0.4989

GaussianNB classifier: precision = 0.9778, recall = 0.3843, accuracy = 0.3944, f1 score = 0.5517

Perceptron classifier: precision = 0.6440, recall = 0.5128, accuracy = 0.6311, f1 score = 0.5710

LinearSVC classifier: precision = 0.2975, recall = 0.5859, accuracy = 0.6521, f1 score = 0.3947

SGDClassifier classifier: precision = 0.3148, recall = 0.4567, accuracy = 0.5961, f1 score = 0.3727

DecisionTreeClassifier classifier: precision = 0.5152, recall = 0.5161, accuracy = 0.6311, f1 score = 0.5157

RandomForestClassifier classifier: precision = 0.4383, recall = 0.6554, accuracy = 0.6981, f1 score = 0.5253

In [ ]:
#missing values in start date
data0 = pd.read_csv("/Users/ying/Bios8366/BIOS_8366_Proj_Rui_Gillian_Ying/Data/cleandata.csv")

dftime = data0[['RUID', 'start_date','end_date']]
dftime.shape #shape is (21252,3), includes 3 columns for each row: ruid, start time and end time, 
#we use these to get the time widow for further filtering
dftime['start_date'].isnull().sum() #there are 82 missing values in start date column
dftime['start_date']=dftime['start_date'].fillna(dftime['end_date'])
#if the start date is missing, i use the end date to fill 



## import ICD9

In [ ]:
#read in icd9
# ICD9

infile7 = "../../Data/FONNESBECK_ICD9_20151202.csv"

df_icd = pd.read_csv(infile7)
df_icd['Event_date'] = pd.to_datetime(df_icd['Event_date'])
df_icd = df_icd.sort_values(['RUID', 'Event_date'])
df_icd['ICD9_Code'].replace({'000.00': None}, inplace = True)

The way I try to deal with ICD9: using counts of individual ICD9 codes
0) build a empty list to store results
1) use dftime to get ruid, start time and end time (time window of that patient)
2) use these information to subset the related icd9 events for this patient and during this time period, store this in a list
3) concatenate all dataframes in the list

In [ ]:
#deal with icd9, similar idea: for each row in dftime (get time window), build a list to store the dataframe generated for each row
icdlist=[]
#n=0

for index, row in dftime.iterrows():    
    mask = (df_icd['RUID'] == row['RUID']) \   #same id 
    & (df_icd['Event_date'] >= row['start_date']) \ #filter by time
    & (df_icd['Event_date'] <= row['end_date'])
    
    new_row=df_icd.loc[mask,] #subset the part of df_icd meeting the criteria above
    
    temp1=new_row.groupby('ICD9_Code').agg('count') #use ICD9 code to group the dataframe, then count occurence of each ICD9 code
    temp2=temp1[['RUID']] #select 1 column (both have same count)
    record=np.empty((1,temp2.shape[0])) #construct empty array to store "transformed" data
    record[0,]=temp2['RUID'].values #keep values in record
    new12=pd.DataFrame(record, columns=temp2.index.tolist()) #construct a dataframe, with values in record and index (ICD9 code) keeped
    
    
    icdlist.append(new12) #use the list to store dataframe generated in each step
    
    #n+=1
    
    #if n>10 : (for testing the code, just ignore)
     #   break

In [ ]:
icdsumm=pd.concat(icdlist,axis=0,join="outer") #outer join the dataframes saved in the list, all values and index keeped
icdsum2=icdsumm.fillna(0) #there are lots of NA generated by outer join, fill with 0 here

## variable reduction of lCD9


Although this looks better than without lab results, we are concerned about using high dimensions of data.
First I wonder what features contribute more. To do it quick and simple, I use the "feature importance"in random forest.

In [ ]:
### variable reduction using PCA, 20 PCs
from sklearn.decomposition import PCA

pca = PCA(n_components=20, whiten=True).fit(icdsum2)
icd_pca = pca.transform(icdsum2)
icd20=pd.DataFrame(icd_pca)

In [ ]:
### variable reduction using PCA, 100 PCs
from sklearn.decomposition import PCA

pca = PCA(n_components=100, whiten=True).fit(icdsum2)
icd_pca = pca.transform(icdsum2)
icd100=pd.DataFrame(icd_pca)

## modeling and evaluation

### combine top 20 PC from ICD9, top 20 PC from lab with all other information

In [ ]:

dflabicd20=pd.concat([df20,icd20],axis=1)

X = dflabicd20.drop(['readmission'], axis = 1).values
y = dflabicd20['readmission'].values

# replace the missing with median
X = Imputer(strategy = 'median').fit_transform(X)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
clfs = {'LogisticRegression':LogisticRegression(),'SVC': SVC(), 
        'KNeighborsClassifier': KNeighborsClassifier(n_neighbors = 3),
        'GaussianNB': GaussianNB(),
        'Perceptron': Perceptron(), 'LinearSVC': LinearSVC(), 
        'SGDClassifier': SGDClassifier(), 
        'DecisionTreeClassifier': DecisionTreeClassifier(),
        'RandomForestClassifier': RandomForestClassifier(n_estimators=100)}

for name, clf in clfs.items():
    clf.fit(X_train,y_train)
    ypred = clf.predict(X_test)
    precision_ = precision_score(ypred, y_test)
    recall_ = recall_score(ypred, y_test)
    accuracy_ = accuracy_score(ypred,y_test)
    f1_ = f1_score(ypred,y_test)
    print('%s classifier: precision = %.4f, recall = %.4f, accuracy = %.4f, f1 score = %.4f' 
          %(name, precision_, recall_, accuracy_, f1_))

performance output:

LogisticRegression classifier: precision = 0.4379, recall = 0.6154, accuracy = 0.6815, f1 score = 0.5117

SVC classifier: precision = 0.4029, recall = 0.6766, accuracy = 0.6990, f1 score = 0.5050

KNeighborsClassifier classifier: precision = 0.5317, recall = 0.5694, accuracy = 0.6683, f1 score = 0.5499

GaussianNB classifier: precision = 0.9621, recall = 0.3952, accuracy = 0.4244, f1 score = 0.5603

Perceptron classifier: precision = 0.5041, recall = 0.5121, accuracy = 0.6280, f1 score = 0.5081

LinearSVC classifier: precision = 0.3885, recall = 0.6215, accuracy = 0.6768, f1 score = 0.4781

SGDClassifier classifier: precision = 0.5498, recall = 0.5072, accuracy = 0.6248, f1 score = 0.5276

DecisionTreeClassifier classifier: precision = 0.5444, recall = 0.5442, accuracy = 0.6526, f1 score = 0.5443

RandomForestClassifier classifier: precision = 0.4481, recall = 0.7179, accuracy = 0.7226, f1 score = 0.5518

### combine top 100 PC from ICD9, top 100 PC from lab with all other information

In [ ]:
dflabicd100=pd.concat([df100,icd100],axis=1)

X = dflabicd100.drop(['readmission'], axis = 1).values
y = dflabicd100['readmission'].values

# replace the missing with median
X = Imputer(strategy = 'median').fit_transform(X)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
clfs = {'LogisticRegression':LogisticRegression(),'SVC': SVC(), 
        'KNeighborsClassifier': KNeighborsClassifier(n_neighbors = 3),
        'GaussianNB': GaussianNB(),
        'Perceptron': Perceptron(), 'LinearSVC': LinearSVC(), 
        'SGDClassifier': SGDClassifier(), 
        'DecisionTreeClassifier': DecisionTreeClassifier(),
        'RandomForestClassifier': RandomForestClassifier(n_estimators=100)}

for name, clf in clfs.items():
    clf.fit(X_train,y_train)
    ypred = clf.predict(X_test)
    precision_ = precision_score(ypred, y_test)
    recall_ = recall_score(ypred, y_test)
    accuracy_ = accuracy_score(ypred,y_test)
    f1_ = f1_score(ypred,y_test)
    print('%s classifier: precision = %.4f, recall = %.4f, accuracy = %.4f, f1 score = %.4f' 
          %(name, precision_, recall_, accuracy_, f1_))

performance output

LogisticRegression classifier: precision = 0.5556, recall = 0.6490, accuracy = 0.7161, f1 score = 0.5987
SVC classifier: precision = 0.5107, recall = 0.6826, accuracy = 0.7230, f1 score = 0.5843
KNeighborsClassifier classifier: precision = 0.5498, recall = 0.5656, accuracy = 0.6675, f1 score = 0.5576
GaussianNB classifier: precision = 0.9309, recall = 0.4004, accuracy = 0.4424, f1 score = 0.5600
Perceptron classifier: precision = 0.6103, recall = 0.5381, accuracy = 0.6518, f1 score = 0.5719
LinearSVC classifier: precision = 0.5407, recall = 0.6486, accuracy = 0.7133, f1 score = 0.5898
SGDClassifier classifier: precision = 0.5453, recall = 0.5439, accuracy = 0.6524, f1 score = 0.5446
DecisionTreeClassifier classifier: precision = 0.5342, recall = 0.5200, accuracy = 0.6346, f1 score = 0.5270
RandomForestClassifier classifier: precision = 0.4107, recall = 0.7301, accuracy = 0.7175, f1 score = 0.5257